In [1]:
import pandas as pd
import numpy as np

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
warnings.simplefilter('ignore')

import sys
sys.path.extend(['../src/'])

from config import *

import matplotlib.pyplot as plt
import seaborn as sns

from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

% matplotlib inline

config = Config()

data = pd.read_excel(config.origin_data_file)

由于文本的问题描述和服务报告使用html语言保存的

&nbsp; 在 html里是空格占位符
<br> 是空标签
\n 是回车换行符

需要使用正则化剔除掉这些特殊字符

In [32]:
import re

def text_clean(content):
    try:
#         result = re.sub(r'[^\u4e00-\u9fa5,A-Za-z0-9]', " ",content)
        result = re.sub('&nbsp;', "",content)
        result = re.sub('<br>', "",result)
#         result = re.sub('正常', " ",result)
        result = re.sub('\n', "",result)
    except:
        result = ''
    return result

In [37]:
for i in range(100):
    print(text_clean(data['问题描述'][i]))

1235仪器偶尔提示TipInAir。
偶尔17-OHP实验提示TipInAir4331
Lis与Leica软件对接问题。
FISH扫描对焦失败。
双标实验F点偏高，400万以上；筛查阳性率低于平时，只有2%-3%；
GSL120黑屏重启，显卡温度高。GSL120年度保养。
1、加样针误报clot，不多，每次实验一两个，但客户不接受；2、洗板机头容易堵（我今天过去检查）
1235产筛实验结果，标准品第四点异常，diff偏差超过50%。
GSP性能检测。
近期第二次发生找不到增强液的报错已安排杨露鉴去检查
1235产筛实验，标准品D点偏低，同时，质控中值也偏低，样本中，只要是4号针加样的样本，都有偏低的倾向，因此需要检查加样针的加样精度。（已安排潘盛过去）
1、洗板机堵孔，昨日多次清洁后正常，今天又堵，清洁无效2、最近两次实验高值质控低出控，第一次原因不明，第二次确认为质控复融问题，科室希望做仪器维护排除仪器问题（1个月前已做年度维护，后更换滤光片和导轨）
杭州市妇产科医院2号1235仪器，联机报错，1号针注射器工作异常
客户反映GSL120加油针不灵活；1235做性能验证，并出具报告；
1.新筛实验有报bloodmissing的报错2.170hp实验缓冲液位置报错tipinair3.实验结果出控，复孔标准品的差异很大.客户希望进行相关性能测试
1235实验过程中突然在去血片过程中出现真空错误，疑似发生严重漏气
近期加样枪两次问题，一次去吸头时的up-downmoversteppingerror，一次去黑帽时的up-downmoversteppingerror，并且提示取不到黑帽；还有长期以来的真空错误
MB8扫描对焦模糊；初步判断为显微镜Z轴磨损，待下周二与奥林帕斯售后仪器上门维修。
KM2明场捕获图像质量不理想，要求调试参数；3台KM系统维护保养；
客户反映最近阳性标本复查重复性下降，cv超过10%的标本增多。需要仪器保养与性能验证。
早孕实验报错“foamdetected”，导致不吸样，加样针涂层损坏严重，需要更换
1235联机报错epprom读取错误，怀疑node1IC14故障需要更换
确认空气过滤器需要更换，销售已报价。
维修洗板机堵，并执行年度维护；
例行执行半年度维护
1235 更换电脑 系统重装软件重装
执行1235半年度维护
执行1235季度维护
G

可以看到问题描述中有很多年度维护，半年度维护，季度维护的描述

In [38]:
data['问题描述_clean'] = data['问题描述'].apply(text_clean)

In [40]:
data['是否定期维护'] = data['问题描述_clean'].apply(lambda x: '度维护' in x)

In [42]:
data.loc[data['是否定期维护'],'问题描述_clean']

11      1、洗板机堵孔，昨日多次清洁后正常，今天又堵，清洁无效2、最近两次实验高值质控低出控，第一次...
23                                        维修洗板机堵，并执行年度维护；
24                                              例行执行半年度维护
26                                            执行1235半年度维护
27                                             执行1235季度维护
32                                               1235年度维护
67                                                按计划年度维护
69                                                按计划年度维护
72                                                按计划年度维护
74                                                按计划年度维护
85                                                按计划年度维护
87                                                按计划年度维护
90                                                按计划年度维护
91      早孕的pappa实验室，1号针在吸取了200ul稀释液后，再去探测样本液面时，容易误报col...
115                                             按计划年度维护保养
129                                               按计划年度维护
135                                               按计划年度维护
136           

可以看到，按照'度维护'关键词提取后，有几种异常情况

1. 存在处理了问题后，进行年度维护的情况
2. 有可能修理的是一台仪器，维护的是另一台仪器

分析的问题在于，如何预测下一次修理可能的时间，定期去维护，维护的主体是仪器还是仪器的部件

In [43]:
data['服务报告_clean'] = data['服务报告'].apply(text_clean)

In [52]:
for i in range(20):
    print(text_clean(data['服务报告'][i]))

1、校准右枪Flag传感器位置；2、对换左右枪；3、待观察
1、校准右枪TipSensor压力；2、移除空气过滤器，真空报错正常；3、待观察；
leica软件生成xml文件，包含病人信息及图像信息；Lis直接调用xml文件内容，实现对接；
1、测试发现客户未调试好曝光时间；2、培训客户调试曝光时间，测试扫描正常；
检查仪器Flash值为1380，反馈滤光片腐蚀；用增强液及无水酒精清洁反馈滤光片；Flash下降至1180；用Eu定标液重新校准FLASH值至1080；测试实验双标F点降低至330万，恢复正常；
问题：GSL120黑屏，怀疑显卡故障，年度维护处理：到场后，客户已更换显卡，电脑能正常工作。根据客户反应，最近一次有一张玻片拍到空白照片。检查后发现仪器光源不稳定，摄像头画面明暗闪烁。询问得知客户最近自行跟换灯泡，检查灯泡发现瓦数不对，正常使用100w的灯泡，客户换上去的是30w换100w灯泡后光源稳定仪器全套校准结果：隔夜扫片正常。
检查确认仪器大概率不存在问题，首先每次实验报clot的数量减少，只有一两个，其次是报错并不集中于某一根针，3次报错分别是三根不同的加样针。进行的操作是检查了加样针的固定和排线的链接，重新校准了液面感应。并对样本上机前处理，同许主任进行了交流。针对浙江等地的经验，即上机前用竹签再挑一下，许主任暂时是抵触的，需要俞勇再去教育一下。客户反应的另一个洗板机头容易堵的情况，检查下来没有发现什么异常，清洁了洗板机头。
对四根加样针进行精度测试：P1=25.33，P2=25.17，P3=25.12，P4=25.23，正常；进行洗板测试，发现部分孔注水不满，吸的时候对应的单数列第四个点吸的水吸不干，残留三分之一左右；实验标准曲线不好是由于洗板机头问题，清洁洗板机头，并指导客户如何清洁；
-washertest正常-MeasurementModuleTest正常-HighVolumePipetteTest正常-LowVolumePipetteTest正常
问题：开机自检报错，机器无法运行；措施：1.查看历史记录，发现实验前未放置稀释杯，导致实验无法往下走2.维护增强液管路，对管路进行疏通，未发现异物结果：1.增强液测试通过；2.开机自检正常；3.跟一次实验正常；
对四根加样针进行精度测试，发现第四根针在加水之后针尖挂水滴；把第四根针与第三根互换，发现

In [4]:
pd.options.display.max_columns = 200

data.head(5)

,服务单号,客户名称,所在省份,客户类型,客户联系人,紧急程度,问题发现日期,问题发现时间,服务部门,事项,项目类别,分项内容,工作方式,问题描述,派单主管,分配工程师,维修总费用,设备编号,设备名称,设备型号,主机序列号,装机日期,维修服务内容,故障描述,服务报告,解决方案,服务工时(小时),上门日期,上门时间,解决日期,解决时间,总结提交日期,任务星级,实际问题描述,回访时间,回访对象,问题是否解决,总体满意度,礼仪/形象,技能水平,沟通能力,服务态度
0,BS-维修201512030011,岳阳市妇幼保健院,NaN,VIP,NaN,NaN,2015-11-27,NaN,综合技术部,NaN,NaN,NaN,NaN,1235仪器偶尔提示&nbsp;Tip&nbsp;In&nbsp;Air。,NaN,康迈,NaN,NaN,NaN,NaN,NaN,NaN,维修,Tip In Air,1、校准右枪Flag&nbsp;传感器位置；\n<br>2、对换左右枪；\n<br>3、待观察,NaN,NaN,NaN,NaN,2015-12-04,NaN,2015-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BS-维修201512140012,娄底市妇幼保健院,NaN,VIP,NaN,NaN,2015-12-01,NaN,综合技术部,NaN,NaN,NaN,NaN,偶尔&nbsp;17-OHP实验提示&nbsp;Tip&nbsp;In&nbsp;Air&n...,NaN,康迈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1、校准右枪Tip&nbsp;Sensor压力；\n<br>2、移除空气过滤器，真空报错正常...,NaN,NaN,NaN,NaN,2015-12-02,NaN,2015-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BS-维修201512160015,山东大学附属生殖医院,NaN,NaN,NaN,NaN,2015-12-04,NaN,综合技术部,NaN,NaN,NaN,NaN,Lis与Leica软件对接问题。\n<br>,NaN,康迈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,leica软件生成xml文件，包含病人信息及图像信息；\n<br>Lis直接调用xml文件内...,NaN,NaN,NaN,NaN,2015-12-10,NaN,2015-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BS-维修201512140013,山东大学齐鲁医院,NaN,V VIP,NaN,NaN,2015-12-07,NaN,综合技术部,NaN,NaN,NaN,NaN,FISH扫描对焦失败。,NaN,康迈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1、测试发现客户未调试好曝光时间；\n<br>2、培训客户调试曝光时间，测试扫描正常；,NaN,NaN,NaN,NaN,2015-12-11,NaN,2015-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BS-维修201512160014,绍兴市妇幼保健院,NaN,VIP,NaN,NaN,2015-12-11,NaN,综合技术部,NaN,NaN,NaN,NaN,双标实验F点偏高，400万以上；\n<br>筛查阳性率低于平时，只有2%-3%；,NaN,康迈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,检查仪器Flash值为1380，反馈滤光片腐蚀；\n<br>用增强液及无水酒精清洁反馈滤光片...,NaN,NaN,NaN,NaN,2015-12-15,NaN,2015-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
